In [ ]:
! pip install pytorch-lightning --quiet
! pip install transformers

     |████████████████████████████████| 1.3MB 9.8MB/s 
     |████████████████████████████████| 1.1MB 31.4MB/s 
     |████████████████████████████████| 890kB 52.0MB/s 
     |████████████████████████████████| 2.9MB 53.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=2d93045de02dc4b232411760d632a6bbeffadefa3f3589befee440b5e38e5050
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.metrics import Accuracy 
from pytorch_lightning.metrics.functional.classification import to_categorical 
from torch.utils.data import random_split
from sklearn.metrics import classification_report , confusion_matrix

import torchtext
from torchtext import  data , vocab
from torchtext.datasets import SST

from transformers import BertTokenizer, BertModel


In [ ]:
class BERTSentimentClassifier(pl.LightningModule):
    def __init__(self , num_classes , learning_rate=1e-5 , weight_decay=1e-5 ):
      super().__init__()
      self.bert = BertModel.from_pretrained('bert-base-uncased')
      self.hidden1 = nn.Linear(self.bert.config.hidden_size , 256)
      self.hidden2 = nn.Linear(256,64)
      self.out = nn.Linear(64, num_classes)
      self.dropout = nn.Dropout(0.1)
      self.learning_rate = learning_rate
      self.weight_decay  = weight_decay
     
    def forward(self,x):
        _,x = self.bert(x)
        x = F.elu(x ,alpha=0.2)
        x = self.dropout(x)
        x = F.relu(self.hidden1(x))
        x = self.dropout(x)
        x = F.relu(self.hidden2(x))
        x = self.out(x)
        return x
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate , weight_decay=self.weight_decay)
        return optimizer
    
    def training_step(self,batch,batch_idx):
        x, y = batch.text, batch.label-1

        logits = self(x)
        loss = F.cross_entropy(logits,y)
        accuracy = Accuracy()
        acc = accuracy(torch.tensor(logits).cpu(),torch.tensor(y).cpu())

        pbar = {'training_acc': acc}

        return {'loss' : loss , 'progress_bar':pbar}
    
    def training_epoch_end(self , train_step_outputs):
        avg_loss = torch.stack([x['loss'] for x in train_step_outputs]).mean()
        avg_acc =  torch.stack([x['progress_bar']['training_acc'] for x in train_step_outputs]).mean()

        tqdm_dict = {'train_loss': avg_loss , 'train_acc': avg_acc}
        return {
                'progress_bar': tqdm_dict,
                'log': {'train_loss': avg_loss , 'train_acc': avg_acc},
                }
    
    def validation_step(self , batch , batch_idx):
        result = self.training_step(batch,batch_idx)
        result['progress_bar']['val_acc'] = result['progress_bar']['training_acc']
        return result
    
    def validation_epoch_end(self , val_step_outputs):
        avg_loss = torch.stack([x['loss'] for x in val_step_outputs]).mean()
        avg_acc =  torch.stack([x['progress_bar']['val_acc'] for x in val_step_outputs]).mean()

        tqdm_dict = {'val_loss': avg_loss , 'val_acc': avg_acc}
        return {
                'progress_bar': tqdm_dict,
                'log': {'val_loss': avg_loss , 'val_acc': avg_acc},
                }
    
    def test_step(self , batch , batch_idx):
        x, y = batch.text, batch.label-1
        logits = self(x)
        loss = F.cross_entropy(logits,y)
        logits = torch.tensor(logits)
        accuracy = Accuracy()
        acc = accuracy(logits.cpu(), torch.tensor(y).cpu())

        pbar = {'test_acc': acc }
        return {'loss' : loss , 'progress_bar':pbar}

    def test_epoch_end(self , test_step_outputs):
        avg_loss = torch.stack([x['loss'] for x in test_step_outputs]).mean()
        avg_acc =  torch.stack([x['progress_bar']['test_acc'] for x in test_step_outputs]).mean()

        tqdm_dict = {'test_loss': avg_loss , 'test_acc': avg_acc}
        return {
                'progress_bar': tqdm_dict,
                'log': {'test_loss': avg_loss , 'test_acc': avg_acc},
                }
    

In [ ]:
  class SSTDataModule(pl.LightningDataModule):
    def __init__(self , batch_size=64 , fine_grained = True , gpu=0):
        super().__init__()
        self.batch_size = batch_size
        self.fine_grained = fine_grained
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
            
        init_token_idx = tokenizer.cls_token_id
        eos_token_idx = tokenizer.sep_token_id
        pad_token_idx = tokenizer.pad_token_id
        unk_token_idx = tokenizer.unk_token_id

        def tokenize_and_cut(sentence):
            tokens = tokenizer.tokenize(sentence)
            max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased'] 
            tokens = tokens[:max_input_length-2]
            return tokens

        self.TEXT = data.Field(batch_first = True,
                        use_vocab = False,
                        tokenize = tokenize_and_cut,
                        preprocessing = tokenizer.convert_tokens_to_ids,
                        init_token = init_token_idx,
                        eos_token = eos_token_idx,
                        pad_token = pad_token_idx,
                        unk_token = unk_token_idx)
        self.LABEL = data.Field(sequential=False)
        self.gpu = gpu
       
        self.num_classes = 3 

        if self.fine_grained : 
            self.num_classes = 5
        
        print(f"-- num classes = {self.num_classes}")
        self.train,self.val, self.test = SST.splits(self.TEXT, self.LABEL , fine_grained=self.fine_grained)
        
    def train_dataloader(self):
        return self.train_iter
    
    def val_dataloader(self):
        return self.val_iter
    
    def test_dataloader(self):
        return self.test_iter

def setup_BERT(self,stage=None):
    if stage is None :
        self.LABEL.build_vocab(self.train)
        print(f"Label : {self.LABEL.vocab.stoi}")
        device = 'cpu'
        if self.gpu > 0 :
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        self.train_iter, self.val_iter , self.test_iter = data.BucketIterator.splits(
            (self.train,self.val, self.test), 
            batch_size=self.batch_size,
            sort_within_batch = True,
            device=device
        )

SSTDataModule.setup = setup_BERT
dm = SSTDataModule(batch_size=32 , gpu=1)
dm.setup() 


-- num classes = 5
downloading trainDevTestTrees_PTB.zip


trainDevTestTrees_PTB.zip: 100%|██████████| 790k/790k [00:00<00:00, 2.53MB/s]


extracting
Label : defaultdict(<function _default_unk_index at 0x7ff831c6b268>, {'<unk>': 0, 'positive': 1, 'negative': 2, 'neutral': 3, 'very positive': 4, 'very negative': 5})


In [ ]:
pl.seed_everything(1234)

hparam = {
        "num_classes" : dm.num_classes ,
        "learning_rate" : 5e-5,
        "weight_decay" : 1e-3,
    }

model = BERTSentimentClassifier(num_classes=hparam["num_classes"],learning_rate=hparam["learning_rate"],weight_decay=hparam["weight_decay"])
# training
trainer = pl.Trainer(gpus=1, precision=16, max_epochs=5,gradient_clip_val=0.3, progress_bar_refresh_rate=50)
trainer.fit(model, dm)
trainer.test(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.

  | Name    | Type      | Params
--------------------------------------
0 | bert    | BertModel | 109 M 
1 | hidden1 | Linear    | 196 K 
2 | hidden2 | Linear    | 16.4 K
3 | out     | Linear    | 325   
4 | dropout | Dropout   | 0     


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The validation_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The {progress_bar:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)
/usr/local/li

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.5308), 'test_loss': tensor(1.3840, device='cuda:0')}
--------------------------------------------------------------------------------



/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The testing_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)


[{'test_acc': 0.5308035612106323, 'test_loss': 1.3839584589004517}]